In [ ]:
from langfuse import Langfuse
import json

# Khởi tạo client
langfuse = Langfuse(
    public_key="pk-lf-463ea5d5-92ce-490e-86e4-67aad0adf333",
    secret_key="sk-lf-8cdd292a-cf36-44b8-a191-e86264696263",
    host="http://localhost:3000",
)

your_session_id = "d94c4bbb-bbc2-4268-bc0a-cc7de6846df8"

# Truy vấn traces và sắp xếp
traces_in_session = langfuse.api.trace.list(session_id=your_session_id, limit=100)
sorted_traces = sorted(traces_in_session.data, key=lambda x: x.timestamp)

chat_history = []

for trace in sorted_traces:
    # 1. Lấy câu trả lời của AI (logic này vẫn đúng)
    ai_answer = (
        trace.output.get("content", "") if isinstance(trace.output, dict) else ""
    )

    # 2. ✅ LOGIC MỚI ĐỂ LẤY CHÍNH XÁC CÂU HỎI CỦA USER
    user_question = ""
    if (
        isinstance(trace.input, list)
        and len(trace.input) > 0
        and isinstance(trace.input[0], dict)
    ):
        prompt_content = trace.input[0].get("content", "")

        # Xử lý các trace về sau có cấu trúc "QUESTION:"
        if "**QUESTION:**" in prompt_content:
            try:
                # Tách lấy phần sau "QUESTION:"
                after_question_marker = prompt_content.split("**QUESTION:**")[1]
                # Tách lấy phần trước "---------------------------------"
                user_question = after_question_marker.split(
                    "---------------------------------"
                )[0].strip()
            except IndexError:
                user_question = ""  # Xử lý nếu split thất bại

        # Xử lý 2 trace đầu tiên có cấu trúc "New User Question:"
        elif "**New User Question:**" in prompt_content:
            try:
                user_question = prompt_content.split("**New User Question:**")[
                    -1
                ].strip()
            except IndexError:
                user_question = ""

    # Thêm cặp hỏi-đáp vào lịch sử nếu có đủ thông tin
    if user_question and ai_answer:
        chat_history.append({"user": user_question, "assistant": ai_answer})

# In ra lịch sử chat đã được làm sạch
print("--- Lịch sử chat đã được làm sạch ---")
print(json.dumps(chat_history, indent=2, ensure_ascii=False))
